# 0.0 Divide your data into distributed train, val and test sets

Our file todo.txt contains all the images and its corresponding classes. Now we want to split our images into train, val and test files, so that the classes are well distributed (e.g. if the dataset is highly imbalanced the images are not randomly separated).

In [1]:
import os

import matplotlib.pylab as plt
import numpy as np
import pandas as pd

# import imgclas
from imgclas import paths, config
# from imgclas.data_utils import load_image, load_data_splits, augment, load_class_names

from sklearn.model_selection import train_test_split

CONF = config.get_conf_dict()

# Customize your image folder if needed
# CONF['general']['images_directory'] = '/media/ignacio/Datos/datasets/semillas/datasets'  # absolute path to image_folder

# splits_dir = paths.get_splits_dir()
splits_dir = '/srv/image-classification-tf/data/dataset_files/'

# Load the data
data = pd.read_csv(splits_dir+'Patients_SPIDER_all.txt', sep="*", header=None, names=['image', 'clase'])

In [2]:
data

,image,clase
0,/srv/datos/ENVIO_OFICIAL/NO_TORTUOSOS_OFICIAL/...,0
1,/srv/datos/ENVIO_OFICIAL/NO_TORTUOSOS_OFICIAL/...,0
2,/srv/datos/ENVIO_OFICIAL/NO_TORTUOSOS_OFICIAL/...,0
3,/srv/datos/ENVIO_OFICIAL/NO_TORTUOSOS_OFICIAL/...,0
4,/srv/datos/ENVIO_OFICIAL/NO_TORTUOSOS_OFICIAL/...,0
...,...,...
396,/srv/datos/ENVIO_OFICIAL/TORTUOSOS_OFICIAL/P53...,1
397,/srv/datos/ENVIO_OFICIAL/TORTUOSOS_OFICIAL/P51...,1
398,/srv/datos/ENVIO_OFICIAL/TORTUOSOS_OFICIAL/P54...,1
399,/srv/datos/ENVIO_OFICIAL/TORTUOSOS_OFICIAL/556...,1


In [3]:
# https://machisnelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms/
from collections import Counter
print(Counter(data.clase))

Counter({0: 201, 1: 200})


In [5]:
# split into train test sets
X_train, X_2, y_train, y_2 = train_test_split(data.image, data.clase, test_size=0.15, random_state=1, stratify=data.clase)
print(Counter(y_train))
print(Counter(y_2))

Counter({1: 170, 0: 170})
Counter({0: 31, 1: 30})


In [8]:
X_test, X_val, y_test, y_val = train_test_split(X_2, y_2, test_size=0.49, random_state=1, stratify=y_2)
print(Counter(y_test))
print(Counter(y_val))

Counter({0: 16, 1: 15})
Counter({1: 15, 0: 15})


In [9]:
train=pd.DataFrame(data={'image': X_train, 'clase': y_train})
train.to_csv(r'/srv/image-classification-tf/data/dataset_files/train.txt', sep=' ', index=None, header=None, mode='a')

In [10]:
val=pd.DataFrame(data={'image': X_val, 'clase': y_val})
val.to_csv(r'/srv/image-classification-tf/data/dataset_files/val.txt', sep=' ', index=None, header=None, mode='a')

In [11]:
test=pd.DataFrame(data={'image': X_test, 'clase': y_test})
test.to_csv(r'/srv/image-classification-tf/data/dataset_files/test.txt', sep=' ', index=None, header=None, mode='a')

In [12]:
len(train)

340

In [13]:
len(test)

31

In [14]:
len(val)

30